# **Assignment 4** 
## Unstructured Data Analytics - Anitesh Barua 
### by Malik Ouda, Noah Placke, Mounika Tarigopula, Sarah Teng, Tim Wong

### **Scraping the data:**

In [ ]:
from google.colab import drive 
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
dir = "/content/drive/Shared drives/Unstructured Data Analytics/Assignment 4"

In [ ]:
natgeo_df = pd.read_csv(f'{dir}/natgeo.csv')

# only taking images just in case the Google Image thing freaks out with a video link
natgeo_df = natgeo_df[natgeo_df['GraphImages/__typename'] == 'GraphImage']
natgeo_df

,GraphImages/__typename,GraphImages/comments_disabled,GraphImages/dimensions/height,GraphImages/dimensions/width,GraphImages/display_url,GraphImages/edge_media_preview_like/count,GraphImages/edge_media_to_caption/edges/0/node/text,GraphImages/edge_media_to_comment/count,GraphImages/gating_info,GraphImages/id,GraphImages/is_video,GraphImages/media_preview,GraphImages/owner/id,GraphImages/shortcode,GraphImages/taken_at_timestamp,GraphImages/thumbnail_resources/0/config_height,GraphImages/thumbnail_resources/0/config_width,GraphImages/thumbnail_resources/0/src,GraphImages/thumbnail_resources/1/config_height,GraphImages/thumbnail_resources/1/config_width,GraphImages/thumbnail_resources/1/src,GraphImages/thumbnail_resources/2/config_height,GraphImages/thumbnail_resources/2/config_width,GraphImages/thumbnail_resources/2/src,GraphImages/thumbnail_resources/3/config_height,GraphImages/thumbnail_resources/3/config_width,GraphImages/thumbnail_resources/3/src,GraphImages/thumbnail_resources/4/config_height,GraphImages/thumbnail_resources/4/config_width,GraphImages/thumbnail_resources/4/src,GraphImages/thumbnail_src,GraphImages/urls/0,GraphImages/username,GraphImages/tags/0,GraphImages/tags/1,GraphImages/tags/2,GraphImages/tags/3,GraphImages/tags/4,GraphImages/video_view_count,GraphImages/urls/1,GraphImages/urls/2,GraphImages/urls/3,GraphImages/urls/4,GraphImages/urls/5,GraphImages/urls/6,GraphImages/urls/7,GraphImages/urls/8,GraphImages/urls/9,GraphImages/tags/5,GraphImages/tags/6,GraphImages/tags/7
0,GraphImage,False,720,1080,https://scontent-dfw5-2.cdninstagram.com/v/t51...,68228,Photo by @nataliekeyssar / Luzerne County Demo...,333,NaN,2434788558063528362,False,ACoc54cHrx0P0oYAnAHJP+Rj/wCvSAdCPxqeN9rD5iqnrw...,787132,CHKG0sij-mq,1604469435,150,150,https://scontent-dfw5-2.cdninstagram.com/v/t51...,240,240,https://scontent-dfw5-2.cdninstagram.com/v/t51...,320,320,https://scontent-dfw5-2.cdninstagram.com/v/t51...,480,480,https://scontent-dfw5-2.cdninstagram.com/v/t51...,640,640,https://scontent-dfw5-2.cdninstagram.com/v/t51...,https://scontent-dfw5-2.cdninstagram.com/v/t51...,https://scontent-dfw5-2.cdninstagram.com/v/t51...,natgeo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GraphImage,False,720,1080,https://scontent-dfw5-2.cdninstagram.com/v/t51...,137945,Photo by @nataliekeyssar / On Election Day Joe...,1542,NaN,2434738352773079979,False,ACocwoXLSKPf3q9cHa2fRP606MAdh+VQ3Lbn2jn5cHHNAF...,787132,CHJ7aHODyOr,1604463450,150,150,https://scontent-dfw5-2.cdninstagram.com/v/t51...,240,240,https://scontent-dfw5-2.cdninstagram.com/v/t51...,320,320,https://scontent-dfw5-2.cdninstagram.com/v/t51...,480,480,https://scontent-dfw5-2.cdninstagram.com/v/t51...,640,640,https://scontent-dfw5-2.cdninstagram.com/v/t51...,https://scontent-dfw5-2.cdninstagram.com/v/t51...,https://scontent-dfw5-2.cdninstagram.com/v/t51...,natgeo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GraphImage,False,1348,1080,https://scontent-dfw5-2.cdninstagram.com/v/t51...,130569,Photo by @nataliekeyssar / Brittany Flores pos...,1118,NaN,2434614106240592382,False,ACEqRFGMjH4dKVXym4AEngAd6XzI41O3kAHjmqwuEjBjbA...,787132,CHJfKFnoCn-,1604448643,150,150,https://scontent-dfw5-2.cdninstagram.com/v/t51...,240,240,https://scontent-dfw5-2.cdninstagram.com/v/t51...,320,320,https://scontent-dfw5-2.cdninstagram.com/v/t51...,480,480,https://scontent-dfw5-2.cdninstagram.com/v/t51...,640,640,https://scontent-dfw5-2.cdninstagram.com/v/t51...,https://scontent-dfw5-2.cdninstagram.com/v/t51...,https://scontent-dfw5-2.cdninstagram.com/v/t51...,natgeo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GraphImage,False,1348,1080,https://scontent-dfw5-2.cdninstagram.com/v/t51...,97058,"Photo by @nataliekeyssar / Allyson Kacmarski, ...",723,NaN,2434584154453575799,False,ACEqfGz5w2Dn0GKiYgDJ/SrSoq9ABVGY5UqOCe/pSGpKOr...,787132,CHJYWO2AAh3,1604445072,150,150,https://scontent-dfw5-2.cdninstagram.com/v/t51...,240,240,https://scont

In [ ]:
natgeo_df_final = natgeo_df[['GraphImages/display_url', 'GraphImages/edge_media_to_caption/edges/0/node/text', 'GraphImages/edge_media_preview_like/count', 'GraphImages/edge_media_to_comment/count']].copy()
natgeo_df_final = natgeo_df_final.rename(columns={'GraphImages/display_url': 'display_url', 'GraphImages/edge_media_to_caption/edges/0/node/text':'caption', 'GraphImages/edge_media_preview_like/count':'likes', 'GraphImages/edge_media_to_comment/count':'comments'})
natgeo_df_final

,display_url,caption,likes,comments
0,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by @nataliekeyssar / Luzerne County Demo...,68228,333
1,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by @nataliekeyssar / On Election Day Joe...,137945,1542
2,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by @nataliekeyssar / Brittany Flores pos...,130569,1118
3,https://scontent-dfw5-2.cdninstagram.com/v/t51...,"Photo by @nataliekeyssar / Allyson Kacmarski, ...",97058,723
4,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by David Guttenfelder @dguttenfelder / F...,77758,424
...,...,...,...,...
794,https://scontent-dfw5-1.cdninstagram.com/v/t51...,"Photo by @nicholesobecki Before the drought, ...",183319,655
795,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by @gerdludwig Mount Demerdzhi near Alu...,388601,1057
796,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by Robin Hammond @Hammond_Robin I was s...,719534,3861
798,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by @lynseyaddario As part of a project ...,144102,605


In [ ]:
!pip install google-cloud-vision

     |████████████████████████████████| 460kB 2.8MB/s 
     |████████████████████████████████| 512kB 13.4MB/s 
     |████████████████████████████████| 92kB 6.5MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 276kB 13.4MB/s 
     |████████████████████████████████| 122kB 16.1MB/s 
  Created wheel for proto-plus: filename=proto_plus-1.11.0-cp36-none-any.whl size=41571 sha256=bed6d5b8c5ac3d29543cfa74c14845cade8f1a58594248bb459f4e37896662c7
  Stored in directory: /root/.cache/pip/wheels/78/e3/6b/a14506581b1cde1ac1743f2939dcc06fc06a5af2aa224a334e
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=99568f04561eb81a2291606bd3d09b060c10efb6ca5df3e5e7987ad17a9cdbef
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built proto-plus pyyaml
ERROR: google-colab 1.0.0 has requirement google-auth~=1.17.2, but you'll have google-auth 1.23.0

In [ ]:
def get_labels(row):
    path = row[0]
    image = vision.Image()
    image.source.image_uri = path
    response = client.label_detection(image=image)
    labels = response.label_annotations
    labels = [label.description for label in labels]
    return labels

In [ ]:
import io
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=f'{dir}/vision-key.json'

# Imports the Google Cloud client library
from google.cloud import vision

# Instantiates a client
client = vision.ImageAnnotatorClient()

natgeo_df_final['labels'] = natgeo_df_final.apply(get_labels, axis=1)
natgeo_df_final.head()

,display_url,caption,likes,comments,labels
0,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by @nataliekeyssar / Luzerne County Demo...,68228,333,"[Night, Audio equipment, Technology, Photograp..."
1,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by @nataliekeyssar / On Election Day Joe...,137945,1542,"[Microphone, Musician, Audio equipment, Guitar..."
2,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by @nataliekeyssar / Brittany Flores pos...,130569,1118,"[People, Pink, Youth, Child, Fun, Photography,..."
3,https://scontent-dfw5-2.cdninstagram.com/v/t51...,"Photo by @nataliekeyssar / Allyson Kacmarski, ...",97058,723,"[Green, Outerwear, Jacket, Headgear, Moustache..."
4,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by David Guttenfelder @dguttenfelder / F...,77758,424,[Event]


In [ ]:
natgeo_df_final.to_csv(f'{dir}/natgeo_labeled.csv')

## **Task A - Engagement Metrics**

In [ ]:
df = pd.read_csv(f'{dir}/natgeo_labeled.csv', index_col=0)
df.head()

,display_url,caption,likes,comments,labels
0,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by @nataliekeyssar / Luzerne County Demo...,68228,333,"['Night', 'Audio equipment', 'Technology', 'Ph..."
1,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by @nataliekeyssar / On Election Day Joe...,137945,1542,"['Microphone', 'Musician', 'Audio equipment', ..."
2,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by @nataliekeyssar / Brittany Flores pos...,130569,1118,"['People', 'Pink', 'Youth', 'Child', 'Fun', 'P..."
3,https://scontent-dfw5-2.cdninstagram.com/v/t51...,"Photo by @nataliekeyssar / Allyson Kacmarski, ...",97058,723,"['Green', 'Outerwear', 'Jacket', 'Headgear', '..."
4,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by David Guttenfelder @dguttenfelder / F...,77758,424,['Event']


In [ ]:
df['normalized_likes'] = df['likes'] / df['likes'].max()
df['normalized_comments'] = df['comments'] / df['comments'].max()
df.head()

,display_url,caption,likes,comments,labels,normalized_likes,normalized_comments
0,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by @nataliekeyssar / Luzerne County Demo...,68228,333,"['Night', 'Audio equipment', 'Technology', 'Ph...",0.061424,0.007095
1,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by @nataliekeyssar / On Election Day Joe...,137945,1542,"['Microphone', 'Musician', 'Audio equipment', ...",0.124189,0.032854
2,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by @nataliekeyssar / Brittany Flores pos...,130569,1118,"['People', 'Pink', 'Youth', 'Child', 'Fun', 'P...",0.117548,0.023820
3,https://scontent-dfw5-2.cdninstagram.com/v/t51...,"Photo by @nataliekeyssar / Allyson Kacmarski, ...",97058,723,"['Green', 'Outerwear', 'Jacket', 'Headgear', '...",0.087379,0.015404
4,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by David Guttenfelder @dguttenfelder / F...,77758,424,['Event'],0.070004,0.009034


In [ ]:
df['engagement'] = (df['normalized_likes'] * .4) + (df['normalized_comments'] * .6)
df.head()

,display_url,caption,likes,comments,labels,normalized_likes,normalized_comments,engagement
0,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by @nataliekeyssar / Luzerne County Demo...,68228,333,"['Night', 'Audio equipment', 'Technology', 'Ph...",0.061424,0.007095,0.028827
1,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by @nataliekeyssar / On Election Day Joe...,137945,1542,"['Microphone', 'Musician', 'Audio equipment', ...",0.124189,0.032854,0.069388
2,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by @nataliekeyssar / Brittany Flores pos...,130569,1118,"['People', 'Pink', 'Youth', 'Child', 'Fun', 'P...",0.117548,0.023820,0.061311
3,https://scontent-dfw5-2.cdninstagram.com/v/t51...,"Photo by @nataliekeyssar / Allyson Kacmarski, ...",97058,723,"['Green', 'Outerwear', 'Jacket', 'Headgear', '...",0.087379,0.015404,0.044194
4,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by David Guttenfelder @dguttenfelder / F...,77758,424,['Event'],0.070004,0.009034,0.033422


In [ ]:
df.drop(['normalized_likes', 'normalized_comments'], axis=1, inplace=True)
df.head()

,display_url,caption,likes,comments,labels,engagement
0,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by @nataliekeyssar / Luzerne County Demo...,68228,333,"['Night', 'Audio equipment', 'Technology', 'Ph...",0.028827
1,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by @nataliekeyssar / On Election Day Joe...,137945,1542,"['Microphone', 'Musician', 'Audio equipment', ...",0.069388
2,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by @nataliekeyssar / Brittany Flores pos...,130569,1118,"['People', 'Pink', 'Youth', 'Child', 'Fun', 'P...",0.061311
3,https://scontent-dfw5-2.cdninstagram.com/v/t51...,"Photo by @nataliekeyssar / Allyson Kacmarski, ...",97058,723,"['Green', 'Outerwear', 'Jacket', 'Headgear', '...",0.044194
4,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by David Guttenfelder @dguttenfelder / F...,77758,424,['Event'],0.033422


In [ ]:
med = df['engagement'].median()
df['engagement_level'] = df['engagement'].apply(lambda e: 1 if e >= med else 0)
df.head()

,display_url,caption,likes,comments,labels,engagement,engagement_level
0,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by @nataliekeyssar / Luzerne County Demo...,68228,333,"['Night', 'Audio equipment', 'Technology', 'Ph...",0.028827,0
1,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by @nataliekeyssar / On Election Day Joe...,137945,1542,"['Microphone', 'Musician', 'Audio equipment', ...",0.069388,0
2,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by @nataliekeyssar / Brittany Flores pos...,130569,1118,"['People', 'Pink', 'Youth', 'Child', 'Fun', 'P...",0.061311,0
3,https://scontent-dfw5-2.cdninstagram.com/v/t51...,"Photo by @nataliekeyssar / Allyson Kacmarski, ...",97058,723,"['Green', 'Outerwear', 'Jacket', 'Headgear', '...",0.044194,0
4,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by David Guttenfelder @dguttenfelder / F...,77758,424,['Event'],0.033422,0


In [ ]:
df.to_csv(f'{dir}/natgeo_engagement.csv')

## **Task B - Logistic Regression**

In [ ]:
df['labels'] = df['labels'].apply(eval)
df.head()

,display_url,caption,likes,comments,labels,engagement,engagement_level
0,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by @nataliekeyssar / Luzerne County Demo...,68228,333,"[Night, Audio equipment, Technology, Photograp...",0.028827,0
1,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by @nataliekeyssar / On Election Day Joe...,137945,1542,"[Microphone, Musician, Audio equipment, Guitar...",0.069388,0
2,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by @nataliekeyssar / Brittany Flores pos...,130569,1118,"[People, Pink, Youth, Child, Fun, Photography,...",0.061311,0
3,https://scontent-dfw5-2.cdninstagram.com/v/t51...,"Photo by @nataliekeyssar / Allyson Kacmarski, ...",97058,723,"[Green, Outerwear, Jacket, Headgear, Moustache...",0.044194,0
4,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by David Guttenfelder @dguttenfelder / F...,77758,424,[Event],0.033422,0


In [ ]:
df['length'] = df['labels'].apply(len)
df.head()

,display_url,caption,likes,comments,labels,engagement,engagement_level,length
0,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by @nataliekeyssar / Luzerne County Demo...,68228,333,"[Night, Audio equipment, Technology, Photograp...",0.028827,0,5
1,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by @nataliekeyssar / On Election Day Joe...,137945,1542,"[Microphone, Musician, Audio equipment, Guitar...",0.069388,0,10
2,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by @nataliekeyssar / Brittany Flores pos...,130569,1118,"[People, Pink, Youth, Child, Fun, Photography,...",0.061311,0,10
3,https://scontent-dfw5-2.cdninstagram.com/v/t51...,"Photo by @nataliekeyssar / Allyson Kacmarski, ...",97058,723,"[Green, Outerwear, Jacket, Headgear, Moustache...",0.044194,0,9
4,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Photo by David Guttenfelder @dguttenfelder / F...,77758,424,[Event],0.033422,0,1


In [ ]:
max_labels = df['length'].max()
max_labels

10

In [ ]:
df_lr = df.copy()

In [ ]:
df_lr.drop(['display_url', 'caption', 'likes', 'comments', 'engagement', 'length'], axis=1, inplace=True)
df_lr.head()

,labels,engagement_level
0,"[Night, Audio equipment, Technology, Photograp...",0
1,"[Microphone, Musician, Audio equipment, Guitar...",0
2,"[People, Pink, Youth, Child, Fun, Photography,...",0
3,"[Green, Outerwear, Jacket, Headgear, Moustache...",0
4,[Event],0


In [ ]:
df_lr = df_lr.drop('labels', 1).join(df_lr.labels.str.join('|').str.get_dummies())

In [ ]:
X, y = df_lr.drop('engagement_level', axis=1), df_lr['engagement_level']
X.shape, y.shape

((625, 1132), (625,))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

lr = LogisticRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.7246376811594203

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[80 26]
 [31 70]]


In [ ]:
df_lr = df.copy()
df_lr.drop(['display_url', 'labels', 'likes', 'comments', 'engagement', 'length'], axis=1, inplace=True)
df_lr.head()

,caption,engagement_level
0,Photo by @nataliekeyssar / Luzerne County Demo...,0
1,Photo by @nataliekeyssar / On Election Day Joe...,0
2,Photo by @nataliekeyssar / Brittany Flores pos...,0
3,"Photo by @nataliekeyssar / Allyson Kacmarski, ...",0
4,Photo by David Guttenfelder @dguttenfelder / F...,0


In [ ]:
X, y = df_lr['caption'], df_lr['engagement_level']
X.shape, y.shape

((625,), (625,))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
text_transformer = TfidfVectorizer(stop_words='english', lowercase=True)
X_train_text = text_transformer.fit_transform(X_train)
X_test_text = text_transformer.transform(X_test)

In [ ]:
X_train_text.shape, X_test_text.shape

((418, 7084), (207, 7084))

In [ ]:
lr = LogisticRegression()
lr.fit(X_train_text, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_pred = lr.predict(X_test_text)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.7004830917874396

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[69 37]
 [25 76]]


In [ ]:
df_lr = df.copy()
df_lr.drop(['display_url', 'likes', 'comments', 'engagement', 'length'], axis=1, inplace=True)
df_lr.head()

,caption,labels,engagement_level
0,Photo by @nataliekeyssar / Luzerne County Demo...,"[Night, Audio equipment, Technology, Photograp...",0
1,Photo by @nataliekeyssar / On Election Day Joe...,"[Microphone, Musician, Audio equipment, Guitar...",0
2,Photo by @nataliekeyssar / Brittany Flores pos...,"[People, Pink, Youth, Child, Fun, Photography,...",0
3,"Photo by @nataliekeyssar / Allyson Kacmarski, ...","[Green, Outerwear, Jacket, Headgear, Moustache...",0
4,Photo by David Guttenfelder @dguttenfelder / F...,[Event],0


In [ ]:
df_lr['labels'] = df_lr.labels.apply(lambda l: ' '.join(l))

In [ ]:
df_lr['combined'] = df_lr['caption'] + ' ' + df_lr['labels']
df_lr.drop(['caption', 'labels'], axis=1, inplace=True)
df_lr.head()

,engagement_level,combined
0,0,Photo by @nataliekeyssar / Luzerne County Demo...
1,0,Photo by @nataliekeyssar / On Election Day Joe...
2,0,Photo by @nataliekeyssar / Brittany Flores pos...
3,0,"Photo by @nataliekeyssar / Allyson Kacmarski, ..."
4,0,Photo by David Guttenfelder @dguttenfelder / F...


In [ ]:
X, y = df_lr.drop('engagement_level', axis=1), df_lr['engagement_level']
X.shape, y.shape

((625, 1), (625,))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
text_transformer = TfidfVectorizer(stop_words='english', lowercase=True)
X_train_text = text_transformer.fit_transform(X_train['combined'])
X_test_text = text_transformer.transform(X_test['combined'])

In [ ]:
X_train_text.shape, X_test_text.shape

((418, 7497), (207, 7497))

In [ ]:
lr = LogisticRegression()
lr.fit(X_train_text, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
y_pred = lr.predict(X_test_text)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.7729468599033816

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[80 26]
 [21 80]]


In [ ]:
df
df_lda = df.copy()
df_lda.drop(['display_url', 'caption', 'likes', 'comments'], axis=1, inplace=True)
df_lda

,labels,engagement,engagement_level
0,"['Night', 'Audio equipment', 'Technology', 'Ph...",0.028827,0
1,"['Microphone', 'Musician', 'Audio equipment', ...",0.069388,0
2,"['People', 'Pink', 'Youth', 'Child', 'Fun', 'P...",0.061311,0
3,"['Green', 'Outerwear', 'Jacket', 'Headgear', '...",0.044194,0
4,['Event'],0.033422,0
...,...,...,...
794,"['Face', 'Red', 'Head', 'Nose', 'Human', 'Eye'...",0.074388,0
795,"['Rock', 'Atmospheric phenomenon', 'Outcrop', ...",0.153452,1
796,"['Face', 'Skin', 'Wrinkle', 'People', 'Forehea...",0.308469,1
798,"['People', 'Vehicle', 'Car', 'Town', 'Street',...",0.059627,0


## **Task C - Topic Modeling**

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

count_vectorizer = CountVectorizer(stop_words='english')

# Fit and transform the processed titles
count_data = count_vectorizer.fit_transform(df_lda['labels'])

 
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
# Tweak the two parameters below
number_topics = 5
number_words = 10
# Create and fit the LDA model
lda = LDA(n_components=number_topics, n_jobs=-1)
lda.fit(count_data)
# Print the topics found by the LDA model
print("Topics found via LDA:")
print_topics(lda, count_vectorizer, number_words)

Topics found via LDA:

Topic #0:
mountain landscape water sky phenomenon natural nature rock tree photography

Topic #1:
night sky architecture vehicle photography light street atmosphere car city

Topic #2:
photography black white monochrome event mammal crowd people seal performance

Topic #3:
plant grass sky tree natural fish grassland marine organism landscape

Topic #4:
wildlife animal bird mammal vertebrate terrestrial ice room cats carnivore


In [ ]:
topic_guess = lda.transform(count_data)
topic_guess

array([[0.02881304, 0.88476263, 0.02920266, 0.02858818, 0.02863349],
       [0.01336878, 0.68713394, 0.27281663, 0.01333609, 0.01334456],
       [0.83420677, 0.01841046, 0.01847874, 0.11020019, 0.01870384],
       ...,
       [0.92707178, 0.0182602 , 0.01827143, 0.01819029, 0.0182063 ],
       [0.01702257, 0.78706786, 0.16253009, 0.01671247, 0.01666701],
       [0.93143772, 0.01693857, 0.01801684, 0.01668742, 0.01691945]])

In [ ]:
topic_df = pd.DataFrame(topic_guess)
topic_df

,0,1,2,3,4
0,0.028813,0.884763,0.029203,0.028588,0.028633
1,0.013369,0.687134,0.272817,0.013336,0.013345
2,0.834207,0.018410,0.018479,0.110200,0.018704
3,0.018347,0.926465,0.018565,0.018270,0.018353
4,0.100246,0.100425,0.598882,0.100446,0.100000
...,...,...,...,...,...
620,0.925993,0.019135,0.018443,0.018231,0.018198
621,0.838584,0.016756,0.016668,0.111312,0.016681
622,0.927072,0.018260,0.018271,0.018190,0.018206
623,0.017023,0.787068,0.162530,0.016712,0.016667


In [ ]:
# df_lda = df_lda.reset_index()
# topic_df = topic_df.reset_index()

df_lda_topic = pd.concat([df_lda, topic_df], axis=1)
df_lda_topic = df_lda_topic.drop(['level_0', 'index'], axis=1)
df_lda_topic

,labels,engagement,engagement_level,0,1,2,3,4
0,"['Night', 'Audio equipment', 'Technology', 'Ph...",0.028827,0,0.028813,0.884763,0.029203,0.028588,0.028633
1,"['Microphone', 'Musician', 'Audio equipment', ...",0.069388,0,0.013369,0.687134,0.272817,0.013336,0.013345
2,"['People', 'Pink', 'Youth', 'Child', 'Fun', 'P...",0.061311,0,0.834207,0.018410,0.018479,0.110200,0.018704
3,"['Green', 'Outerwear', 'Jacket', 'Headgear', '...",0.044194,0,0.018347,0.926465,0.018565,0.018270,0.018353
4,['Event'],0.033422,0,0.100246,0.100425,0.598882,0.100446,0.100000
...,...,...,...,...,...,...,...,...
620,"['Face', 'Red', 'Head', 'Nose', 'Human', 'Eye'...",0.074388,0,0.925993,0.019135,0.018443,0.018231,0.018198
621,"['Rock', 'Atmospheric phenomenon', 'Outcrop', ...",0.153452,1,0.838584,0.016756,0.016668,0.111312,0.016681
622,"['Face', 'Skin', 'Wrinkle', 'People', 'Forehea...",0.308469,1,0.927072,0.018260,0.018271,0.018190,0.018206
623,"['People', 'Vehicle', 'Car', 'Town', 'Street',...",0.059627,0,0.017023,0.787068,0.162530,0.016712,0.016667


In [ ]:
low, high = df_lda_topic.engagement.quantile([0.25,.75])

df_low = df_lda_topic.loc[(df_lda_topic.engagement < low)]
df_high = df_lda_topic.loc[(df_lda_topic.engagement > high)]
# df_lda_topic.query('{low}<engagement<{high}'.format(low=low,high=high))
df_low

,labels,engagement,engagement_level,0,1,2,3,4
0,"['Night', 'Audio equipment', 'Technology', 'Ph...",0.028827,0,0.028813,0.884763,0.029203,0.028588,0.028633
2,"['People', 'Pink', 'Youth', 'Child', 'Fun', 'P...",0.061311,0,0.834207,0.018410,0.018479,0.110200,0.018704
3,"['Green', 'Outerwear', 'Jacket', 'Headgear', '...",0.044194,0,0.018347,0.926465,0.018565,0.018270,0.018353
4,['Event'],0.033422,0,0.100246,0.100425,0.598882,0.100446,0.100000
5,"['Table', 'Design', 'Room', 'Event', 'Adaptati...",0.026277,0,0.028816,0.028695,0.666720,0.028652,0.247116
...,...,...,...,...,...,...,...,...
601,"['Black', 'White', 'Black-and-white', 'Photogr...",0.058247,0,0.015497,0.015609,0.938123,0.015386,0.015385
606,"['Facial hair', 'Human', 'Beard', 'Room', 'Pho...",0.063797,0,0.022745,0.265076,0.265925,0.022251,0.424003
612,"['Artist', 'Sculptor']",0.055455,0,0.066674,0.733291,0.066676,0.066680,0.066680
619,"['People', 'Event', 'Festival', 'Crowd', 'Publ...",0.061556,0,0.016964,0.017096,0.932573,0.016700,0.016667


In [ ]:
df_high

,labels,engagement,engagement_level,0,1,2,3,4
11,"['Grass', 'Sky', 'Lawn', 'Grassland', 'Cloud',...",0.264540,1,0.018349,0.202729,0.018579,0.561645,0.198698
14,"['Flower', 'Flowering plant', 'Lupin', 'Plant'...",0.128577,1,0.016933,0.016686,0.016753,0.932924,0.016704
17,"['Wildlife', 'Nature', 'Sky', 'Canidae', 'Wild...",0.161614,1,0.159630,0.016907,0.276729,0.017105,0.529629
30,"['Vertebrate', 'Bird', 'Bird of prey', 'Accipi...",0.173822,1,0.014286,0.014286,0.014319,0.014303,0.942806
31,"['Event', 'Architecture', 'Performance', 'Phot...",0.406873,1,0.033447,0.432051,0.467749,0.033414,0.033340
...,...,...,...,...,...,...,...,...
615,"['Iceberg', 'Polar ice cap', 'Ice', 'Sea ice',...",0.167363,1,0.343818,0.011117,0.011141,0.011158,0.622765
617,"['Tree', 'Bigtree', 'Nature', 'Trunk', 'Old-gr...",0.157005,1,0.728098,0.012536,0.233886,0.012980,0.012501
621,"['Rock', 'Atmospheric phenomenon', 'Outcrop', ...",0.153452,1,0.838584,0.016756,0.016668,0.111312,0.016681
622,"['Face', 'Skin', 'Wrinkle', 'People', 'Forehea...",0.308469,1,0.927072,0.018260,0.018271,0.018190,0.018206


In [ ]:
df_low.describe()

,engagement,engagement_level,0,1,2,3,4
count,156.000000,156.0,156.000000,156.000000,156.000000,156.000000,156.000000
mean,0.054992,0.0,0.212796,0.284754,0.275095,0.130820,0.096536
std,0.008271,0.0,0.315791,0.343770,0.342542,0.249174,0.202053
min,0.026277,0.0,0.011118,0.011783,0.011200,0.011120,0.011203
25%,0.049278,0.0,0.016799,0.018357,0.018236,0.015689,0.015499
50%,0.057549,0.0,0.025475,0.054732,0.033598,0.020241,0.019804
75%,0.061257,0.0,0.325090,0.558938,0.538825,0.066817,0.042551
max,0.064986,0.0,0.949757,0.955361,0.952893,0.952692,0.942741


In [ ]:
df_low_mean = df_low.describe().iloc[1, 2:]
df_low_mean

0    0.212796
1    0.284754
2    0.275095
3    0.130820
4    0.096536
Name: mean, dtype: float64

In [ ]:
df_high.describe()

,engagement,engagement_level,0,1,2,3,4
count,156.000000,156.0,156.000000,156.000000,156.000000,156.000000,156.000000
mean,0.196651,1.0,0.267624,0.193837,0.158822,0.151381,0.228336
std,0.088064,0.0,0.338542,0.312768,0.290277,0.276223,0.343464
min,0.127520,1.0,0.008059,0.008014,0.011132,0.008164,0.008023
25%,0.144595,1.0,0.014630,0.014286,0.014357,0.014304,0.014429
50%,0.166214,1.0,0.022893,0.016831,0.016679,0.015656,0.016832
75%,0.208060,1.0,0.505267,0.215688,0.036436,0.102832,0.376668
max,0.733834,1.0,0.949743,0.954590,0.967739,0.949920,0.952857


In [ ]:
df_high_mean = df_high.describe().iloc[1, 2:]
df_high_mean

0    0.267624
1    0.193837
2    0.158822
3    0.151381
4    0.228336
Name: mean, dtype: float64

In [ ]:
means = pd.DataFrame([df_low_mean, df_high_mean], index = ['low', 'high'])
means

,0,1,2,3,4
low,0.212796,0.284754,0.275095,0.130820,0.096536
high,0.267624,0.193837,0.158822,0.151381,0.228336


## **Task D - Recommendations**

What advice would you give National Geographic if it wants to increase engagement on its Instagram page based on your findings in Tasks B and C?

If National Geographic wants to increase engagement on their Instagram page, we recommend that they post more content related to topic #4, which in our case is photos relating to wildlife animals (‘wildlife, animal, bird, mammal, vertebrate, terrestrial, ice, room, cats, carnivore’). Based on our analysis in task B, we found that these topics were more clearly defined when we combined the post captions with the image labels (the accuracy score increased when we combined the two). Based on our analysis task C, we were able to calculate the weights assigned to each topic for each photo that we scraped. In the last table of our notebook, we are showcasing the mean weights for each topic on the highest and lowest quartiles of engagement scores for our dataset. In this table, we can see that topic #0 was a high engagement topic for both the high and low engagement posts. However, we noticed that topic #4 scored on high engagement within the “high” engagement score quartile, but low within the “low” engagement score quartile, meaning that this topic is most likely to perform on a high level, rather than be equally engaging on all levels. 

To implement this recommendation, we suggest that National Geographic posts more photos of wildlife and animals, because these are the photos that perform the best and garner the highest engagement scores. Based on a preliminary visual analysis of their Instagram page, we were able to observe this trend first-hand, because many of their photos in-feed that showcase animals/wildlife have many more likes and comments than the photos in their feed that showcase humans or modern-day settings. We believe this also makes sense logically, because National Geographic is typically known for its exquisite photography in nature and wildlife, so this is what their followers expect to see on their Instagram feed. 
